# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 4:29PM,247204,12,HC-8565082,HealthCaps LLC,Executing
1,Sep 23 2022 3:46PM,247195,20,8559095,"ACI Healthcare USA, Inc.",Released
2,Sep 23 2022 3:46PM,247195,20,8559657,"ACI Healthcare USA, Inc.",Released
3,Sep 23 2022 2:56PM,247190,10,0086005917,ISDIN Corporation,Released
4,Sep 23 2022 2:56PM,247190,10,0086005930,ISDIN Corporation,Released
5,Sep 23 2022 2:56PM,247190,10,0086005928,ISDIN Corporation,Released
6,Sep 23 2022 2:56PM,247182,21,627956,"NBTY Global, Inc.",Executing
7,Sep 23 2022 2:55PM,247183,10,MSP214243,"Methapharm, Inc.",Released
8,Sep 23 2022 2:55PM,247183,10,MSP214245,"Methapharm, Inc.",Released
9,Sep 23 2022 2:55PM,247183,10,MSP214246,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
9,247183,Released,20
10,247185,Released,2
11,247190,Released,3
12,247195,Released,2
13,247204,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247183,NaN,NaN,20.0
247185,NaN,NaN,2.0
247190,NaN,NaN,3.0
247195,NaN,NaN,2.0
247204,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247161,0.0,0.0,4.0
247162,0.0,0.0,7.0
247163,0.0,1.0,0.0
247168,3.0,0.0,1.0
247171,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247161,0,0,4
247162,0,0,7
247163,0,1,0
247168,3,0,1
247171,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247161,0,0,4
1,247162,0,0,7
2,247163,0,1,0
3,247168,3,0,1
4,247171,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247161,,,4
1,247162,,,7
2,247163,,1,
3,247168,3,,1
4,247171,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 4:29PM,247204,12,HealthCaps LLC
1,Sep 23 2022 3:46PM,247195,20,"ACI Healthcare USA, Inc."
3,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation
6,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc."
7,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc."
27,Sep 23 2022 2:55PM,247185,10,"Methapharm, Inc."
29,Sep 23 2022 2:46PM,247179,10,"Methapharm, Inc."
35,Sep 23 2022 2:17PM,247175,18,Innova Softgel LLC
36,Sep 23 2022 2:09PM,247171,20,"ACI Healthcare USA, Inc."
37,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 4:29PM,247204,12,HealthCaps LLC,,1,
1,Sep 23 2022 3:46PM,247195,20,"ACI Healthcare USA, Inc.",,,2
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,,,3
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",,1,
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",,,20
5,Sep 23 2022 2:55PM,247185,10,"Methapharm, Inc.",,,2
6,Sep 23 2022 2:46PM,247179,10,"Methapharm, Inc.",,,6
7,Sep 23 2022 2:17PM,247175,18,Innova Softgel LLC,,,1
8,Sep 23 2022 2:09PM,247171,20,"ACI Healthcare USA, Inc.",,,1
9,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc.",3,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 4:29PM,247204,12,HealthCaps LLC,,1,
1,Sep 23 2022 3:46PM,247195,20,"ACI Healthcare USA, Inc.",2,,
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,3,,
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",,1,
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",20,,
5,Sep 23 2022 2:55PM,247185,10,"Methapharm, Inc.",2,,
6,Sep 23 2022 2:46PM,247179,10,"Methapharm, Inc.",6,,
7,Sep 23 2022 2:17PM,247175,18,Innova Softgel LLC,1,,
8,Sep 23 2022 2:09PM,247171,20,"ACI Healthcare USA, Inc.",1,,
9,Sep 23 2022 1:42PM,247168,20,"ACI Healthcare USA, Inc.",1,,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 4:29PM,247204,12,HealthCaps LLC,,1,
1,Sep 23 2022 3:46PM,247195,20,"ACI Healthcare USA, Inc.",2,,
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,3,,
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",,1,
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 4:29PM,247204,12,HealthCaps LLC,NaN,1.0,NaN
1,Sep 23 2022 3:46PM,247195,20,"ACI Healthcare USA, Inc.",2.0,NaN,NaN
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,3.0,NaN,NaN
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",NaN,1.0,NaN
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 4:29PM,247204,12,HealthCaps LLC,0.0,1.0,0.0
1,Sep 23 2022 3:46PM,247195,20,"ACI Healthcare USA, Inc.",2.0,0.0,0.0
2,Sep 23 2022 2:56PM,247190,10,ISDIN Corporation,3.0,0.0,0.0
3,Sep 23 2022 2:56PM,247182,21,"NBTY Global, Inc.",0.0,1.0,0.0
4,Sep 23 2022 2:55PM,247183,10,"Methapharm, Inc.",20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1235899,38.0,0.0,0.0
12,247204,0.0,1.0,0.0
16,494324,4.0,1.0,0.0
18,247175,1.0,0.0,0.0
20,741534,4.0,0.0,3.0
21,247182,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1235899,38.0,0.0,0.0
1,12,247204,0.0,1.0,0.0
2,16,494324,4.0,1.0,0.0
3,18,247175,1.0,0.0,0.0
4,20,741534,4.0,0.0,3.0
5,21,247182,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,38.0,0.0,0.0
1,12,0.0,1.0,0.0
2,16,4.0,1.0,0.0
3,18,1.0,0.0,0.0
4,20,4.0,0.0,3.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,12,Released,0.0
2,16,Released,4.0
3,18,Released,1.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,3.0,0.0
Executing,0.0,1.0,1.0,0.0,0.0,1.0
Released,38.0,0.0,4.0,1.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,20,21
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,0.0,1.0,1.0,0.0,0.0,1.0
2,Released,38.0,0.0,4.0,1.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,20,21
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,0.0,1.0,1.0,0.0,0.0,1.0
2,Released,38.0,0.0,4.0,1.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()